In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import seaborn as sns

In [2]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(11)

In [43]:
train_features = np.load('../Data/UCI_train_features.npy')
test_features = np.load('../Data/UCI_test_features.npy')
train_labels = np.load('../Data/UCI_train_labels.npy')
test_labels = np.load('../Data/UCI_test_labels.npy')

In [44]:
train_labels = train_labels - 1
test_labels = test_labels - 1

In [46]:
train_labels[:100]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [47]:
tensor_train_features = torch.Tensor(train_features)
tensor_train_labels = torch.Tensor(train_labels)
tensor_test_features = torch.Tensor(test_features)
tensor_test_labels = torch.Tensor(test_labels)

# Add one dimension of channel
tensor_train_features = torch.unsqueeze(tensor_train_features, 1)
tensor_test_features = torch.unsqueeze(tensor_test_features, 1)
train_dataset = TensorDataset(tensor_train_features, tensor_train_labels)
test_dataset = TensorDataset(tensor_test_features, tensor_test_labels)

In [48]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=16,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=16,shuffle=True)

In [20]:
print(len(train_loader))

460


In [59]:
# num_feature = 9
# feature_width = 128
# kernel_size = 2


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU())
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(int(4 * 64 * 32), 120)
        self.fc2 = nn.Linear(120, 6)
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

epoches = 7
lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=lr, weight_decay=0.0001)

In [67]:
loss_list = []
for epoch in range(epoches):
    acc = 0
    for i, data in enumerate(train_loader):
        features, labels = data
        optimizer.zero_grad()
        outputs = model(features.to(device))
        loss = criterion(outputs, labels.to(device).long())
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
        
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.to(device).long()).sum().item()
        acc = correct / total
    print("Epoch:",epoch+1, "accuracy:", acc)

Epoch: 1 accuracy: 0.875
Epoch: 2 accuracy: 0.375
Epoch: 3 accuracy: 0.5
Epoch: 4 accuracy: 0.625
Epoch: 5 accuracy: 1.0
Epoch: 6 accuracy: 0.875
Epoch: 7 accuracy: 0.875


In [63]:
model.train()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=8192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=6, bias=True)
)